<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a1a7005b4ea53eded4a9740a481932cb97dd23690395e992b8eba0ec3ba5f107
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 12:27:24
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.42 C
-------------------
Today PnL: -72.44 K (-0.51%)
Current PnL: -22.34 L (-14.64%)
CY Booked + Current PnL: -9.69 L (-6.35%)
-------------------
Total profit:  1.82 L
Total loss:  -24.17 L
-------------------
Total Booked + Current PnL: 17.29 L (13.76%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.89%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 89.33 L (62.81%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.26%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,68.0,X-LC,6.13,233392.0,23026.0,5765.0,1.56,10.95,2.47,13.68,37.0,3.99,1.65,29.28,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-5.43,65.0,M-LC,11.75,171994.0,31154.0,13330.0,0.75,22.12,7.75,31.59,66.0,2.34,1.22,54.47,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.20,53.0,M-SC,1.94,87521.0,-13256.0,13347.0,0.72,-13.15,15.25,0.09,245.0,-0.99,0.62,14.98,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-11.66,36.0,X-MC,2.86,87180.0,-3498.0,18404.0,-1.34,-3.86,21.11,16.44,101.0,-0.19,0.62,15.54,X40,ATH,PHARMA
33,ICICIGI,2252.93,-14.07,55.0,X-MC,5.80,147608.0,11475.0,19115.0,-0.31,8.43,12.95,22.47,91.0,0.60,1.05,22.50,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VOLTAS,1530.0,-5.83,35.0,X-MC,4.38,197505.0,5763.0,31996.0,1.88,3.01,16.20,19.69,99.0,0.18,1.40,10.01,XY25,NTT,AC
64,SFL,1287.0,33.36,49.0,M-SC,3.61,176175.0,-86043.0,155862.0,1.69,-32.81,88.47,26.63,229.0,-0.55,1.25,9.54,XY24,NTT,MISC
58,RELIANCE,1533.0,-6.91,68.0,X-LC,6.13,233392.0,23026.0,5765.0,1.56,10.95,2.47,13.68,37.0,3.99,1.65,29.28,XY25,NTT,REFINERIES
82,VALIANTORG,838.0,-284.65,42.0,H-SC,11.28,102663.0,-60942.0,182258.0,1.31,-37.25,177.53,74.15,139.0,-0.33,0.73,31.60,XR,NTT,CHEMICALS
35,IEX,219.0,-33.63,47.0,H-SC,16.00,189871.0,-10083.0,107315.0,1.29,-5.04,56.52,48.63,115.0,-0.09,1.35,5.93,XR,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SYMPHONY,1306.0,-45.13,41.0,M-SC,16.76,116754.0,-54387.0,54337.0,-5.06,-31.78,46.54,-0.03,196.0,-1.00,0.83,1.79,OX40N,NTT,DURABLES
70,SURYODAY,240.0,57.89,44.0,H-SC,15.79,145238.0,-33833.0,99561.0,-4.53,-18.89,68.55,36.71,135.0,-0.34,1.03,44.10,XR,NTT,BANKS
1,ABB,7934.0,-42.59,35.0,H-LC,5.10,240072.0,-21547.0,140754.0,-4.33,-8.24,58.63,45.57,7.0,-0.15,1.70,2.15,AR,NTT,ELECTRICAL
19,COFFEEDAY,80.0,-51.71,32.0,L-SC,27.46,72768.0,-40781.0,80831.0,-3.34,-35.91,111.08,35.27,268.0,-0.50,0.52,78.10,XR,NTT,HOTELS
17,CERA,9475.0,-24.11,43.0,H-SC,1.83,138322.0,-37581.0,79604.0,-2.76,-21.36,57.55,23.89,149.0,-0.47,0.98,19.46,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,26.6,46.0,M-LC,10.5,205907.0,831.0,111746.0,-0.0,0.41,54.27,54.89,52.0,0.01,1.46,24.6,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.20,53.0,M-SC,1.94,87521.0,-13256.0,13347.0,0.72,-13.15,15.25,0.09,245.0,-0.99,0.62,14.98,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,45.0,H-SC,2.34,222012.0,-47655.0,83987.0,-0.60,-17.67,37.83,13.47,138.0,-0.57,1.57,12.89,XY24,NTT,PAINTS
17,CERA,9475.0,-24.11,43.0,H-SC,1.83,138322.0,-37581.0,79604.0,-2.76,-21.36,57.55,23.89,149.0,-0.47,0.98,19.46,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19943.48,63.0,M-SC,1.51,97944.0,-29861.0,29844.0,0.06,-23.36,30.47,-0.01,232.0,-1.00,0.69,10.91,OX40N,NTT,AC
67,SIS,528.0,1990.16,41.0,H-SC,2.54,84341.0,-26691.0,50301.0,-2.55,-24.04,59.64,21.26,156.0,-0.53,0.60,13.95,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,65.0,M-LC,11.75,171994.0,31154.0,13330.0,0.75,22.12,7.75,31.59,66.0,2.34,1.22,54.47,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,50.76,60.0,M-MC,10.29,237464.0,12502.0,155159.0,-0.01,5.56,65.34,74.53,192.0,0.08,1.68,39.60,XY24,BTT,STEEL
31,HINDZINC,730.22,26.60,46.0,M-LC,10.50,205907.0,831.0,111746.0,-0.00,0.41,54.27,54.89,52.0,0.01,1.46,24.60,X5K,ATH,METALS
46,KPIGREEN,731.05,7.49,55.0,H-SC,8.03,128104.0,2807.0,56122.0,-0.47,2.24,43.81,47.03,141.0,0.05,0.91,60.40,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,65.0,M-LC,11.75,171994.0,31154.0,13330.0,0.75,22.12,7.75,31.59,66.0,2.34,1.22,54.47,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-41.67,48.0,M-SC,6.13,121044.0,6246.0,87793.0,-2.56,5.44,72.53,81.92,223.0,0.07,0.86,43.30,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-41.67,48.0,M-SC,6.13,121044.0,6246.0,87793.0,-2.56,5.44,72.53,81.92,223.0,0.07,0.86,43.30,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,51.0,H-SC,5.40,129278.0,5942.0,125684.0,-1.37,4.82,97.22,106.72,119.0,0.05,0.92,29.32,AR,ATH,MISC
46,KPIGREEN,731.05,7.49,55.0,H-SC,8.03,128104.0,2807.0,56122.0,-0.47,2.24,43.81,47.03,141.0,0.05,0.91,60.40,MH,ATH,POWER
31,HINDZINC,730.22,26.60,46.0,M-LC,10.50,205907.0,831.0,111746.0,-0.00,0.41,54.27,54.89,52.0,0.01,1.46,24.60,X5K,ATH,METALS
86,WIPRO,420.00,-16.52,39.0,M-LC,5.42,146940.0,-4005.0,113467.0,-1.27,-2.65,77.22,72.51,53.0,-0.04,1.04,3.21,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,32.34,148050.0,-128473.0,238553.0,0.33,-46.46,161.13,39.81,54.0,-0.54,1.05,4.35,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.72,24.0,X-SC,7.78,87150.0,-41520.0,86819.0,-0.07,-32.27,99.62,35.20,219.0,-0.48,0.62,0.03,X40,NTT,FOOTWEAR
23,DMART,5391.45,-20.90,26.0,X-LC,7.55,201505.0,-8902.0,68068.0,-1.28,-4.23,33.78,28.12,38.0,-0.13,1.43,18.39,X40N,ATH,FMCG
44,JSWINFRA,342.00,-23.34,30.0,X-MC,4.57,192375.0,-7972.0,38475.0,0.39,-3.98,20.00,15.23,178.0,-0.21,1.36,26.99,X40N,NTT,REALTY
52,PGHH,17907.65,-32.00,32.0,X-MC,0.17,196905.0,-3915.0,71713.0,-0.27,-1.95,36.42,33.76,80.0,-0.05,1.39,3.15,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-32.00,32.0,X-MC,0.17,196905.0,-3915.0,71713.0,-0.27,-1.95,36.42,33.76,80.0,-0.05,1.39,3.15,X40,ATH,FMCG
41,ITC,452.00,-38.77,43.0,X-LC,1.07,198387.0,-1751.0,22636.0,-0.44,-0.87,11.41,10.44,4.0,-0.08,1.41,4.64,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.39,38.0,H-LC,1.39,152200.0,-33895.0,81381.0,-1.28,-18.21,53.47,25.51,15.0,-0.42,1.08,12.16,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.30,39.0,X-MC,3.19,305781.0,-22210.0,128734.0,0.96,-6.77,42.10,32.48,92.0,-0.17,2.17,3.88,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.52,64.0,X-LC,3.97,245066.0,10922.0,37691.0,0.59,4.66,15.38,20.77,86.0,0.29,1.74,11.91,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.72,24.0,X-SC,7.78,87150.0,-41520.0,86819.0,-0.07,-32.27,99.62,35.20,219.0,-0.48,0.62,0.03,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.41,33.0,X-SC,37.42,49093.0,-15913.0,166838.0,0.47,-24.48,339.84,232.18,198.0,-0.10,0.35,0.48,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.72,32.0,X-MC,7.42,79330.0,-4180.0,24212.0,-0.24,-5.01,30.52,23.98,82.0,-0.17,0.56,0.52,X40,ATH,APPARELS
20,COLPAL,3726.84,-7.18,39.0,X-MC,7.29,216540.0,-46825.0,156147.0,-0.38,-17.78,72.11,41.51,84.0,-0.30,1.53,1.62,XY25,ATH,FMCG
1,ABB,7934.00,-42.59,35.0,H-LC,5.10,240072.0,-21547.0,140754.0,-4.33,-8.24,58.63,45.57,7.0,-0.15,1.70,2.15,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,33.0,X-SC,37.42,49093.0,-15913.0,166838.0,0.47,-24.48,339.84,232.18,198.0,-0.10,0.35,0.48,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.72,32.0,X-MC,7.42,79330.0,-4180.0,24212.0,-0.24,-5.01,30.52,23.98,82.0,-0.17,0.56,0.52,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.56,41.0,X-SC,4.22,81084.0,-64076.0,140000.0,-1.40,-44.14,172.66,52.31,136.0,-0.46,0.57,8.54,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.66,36.0,X-MC,2.86,87180.0,-3498.0,18404.0,-1.34,-3.86,21.11,16.44,101.0,-0.19,0.62,15.54,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.72,24.0,X-SC,7.78,87150.0,-41520.0,86819.0,-0.07,-32.27,99.62,35.20,219.0,-0.48,0.62,0.03,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.65,44.0,X-LC,9.73,279857.0,-66099.0,132792.0,-1.12,-19.11,47.45,19.28,1.0,-0.50,1.98,3.46,X40,ATH,IT
39,INFY,2275.00,-19.96,48.0,X-LC,4.99,312237.0,-651.0,172324.0,-0.05,-0.21,55.19,54.87,3.0,-0.00,2.21,8.06,X40,BTT,IT
41,ITC,452.00,-38.77,43.0,X-LC,1.07,198387.0,-1751.0,22636.0,-0.44,-0.87,11.41,10.44,4.0,-0.08,1.41,4.64,X40,NTT,FMCG
83,VBL,671.64,-16.73,53.0,X-LC,6.33,299924.0,-15918.0,128577.0,-0.37,-5.04,42.87,35.67,5.0,-0.12,2.12,8.12,X40N,ATH,FMCG
1,ABB,7934.00,-42.59,35.0,H-LC,5.10,240072.0,-21547.0,140754.0,-4.33,-8.24,58.63,45.57,7.0,-0.15,1.70,2.15,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-51.71,32.0,L-SC,27.46,72768.0,-40781.0,80831.0,-3.34,-35.91,111.08,35.27,268.0,-0.50,0.52,78.10,XR,NTT,HOTELS
7,ASIANTILES,137.00,7333.33,62.0,L-SC,12.29,81544.0,-12266.0,88883.0,-2.25,-13.08,109.00,81.67,269.0,-0.14,0.58,57.91,XR,NTT,CERAMICS
49,MASFIN,398.61,-20.05,46.0,H-SC,8.31,90675.0,-7305.0,28907.0,-0.64,-7.46,31.88,22.05,152.0,-0.25,0.64,31.80,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-41.67,48.0,M-SC,6.13,121044.0,6246.0,87793.0,-2.56,5.44,72.53,81.92,223.0,0.07,0.86,43.30,XR,NTT,DURABLES
46,KPIGREEN,731.05,7.49,55.0,H-SC,8.03,128104.0,2807.0,56122.0,-0.47,2.24,43.81,47.03,141.0,0.05,0.91,60.40,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,72.45,63.0,H-SC,3.97,149742.0,-33033.0,144097.0,-0.17,-18.07,96.23,60.77,125.0,-0.23,1.06,34.85,XR,NTT,JEWELLERY
46,KPIGREEN,731.05,7.49,55.0,H-SC,8.03,128104.0,2807.0,56122.0,-0.47,2.24,43.81,47.03,141.0,0.05,0.91,60.40,MH,ATH,POWER
68,SONACOMS,806.63,-31.80,67.0,M-SC,6.45,84866.0,-16293.0,56292.0,0.42,-16.11,66.33,39.54,202.0,-0.29,0.60,19.67,AR,ATH,AUTO
62,SAMMAANCAP,326.00,-196.59,69.0,M-SC,34.00,167373.0,18153.0,126032.0,0.36,12.17,75.30,96.62,208.0,0.14,1.19,81.13,XY25,NTT,FINANCE
7,ASIANTILES,137.00,7333.33,62.0,L-SC,12.29,81544.0,-12266.0,88883.0,-2.25,-13.08,109.00,81.67,269.0,-0.14,0.58,57.91,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.71
1,25,44.32
2,50,75.51


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.77
LC    33.17
MC    23.10
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.95
X40      16.29
X40N     12.46
XY25     11.97
XR        9.86
AR        7.81
OX40N     6.52
X200      1.77
X5K       1.46
SR        1.04
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.24
X-LC    22.50
X-MC    18.77
M-SC    12.59
X-SC     5.51
M-LC     5.14
H-LC     4.55
H-MC     2.33
M-MC     1.68
L-SC     1.43
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.29,-4.76,38.63
IT,12.45,-19.76,83.92
FINANCE,12.14,-10.26,60.24
MISC,7.80,-18.32,78.00
ELECTRICAL,5.72,-12.10,51.84
PAINTS,5.60,-16.80,34.20
BANKS,4.03,-18.85,87.99
INSURANCE,3.89,-0.88,38.50
AUTO,2.72,-49.60,112.04


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3346301.0,22
XR,1267135.0,13
AR,1186066.0,9
X40,858741.0,12
X40N,709780.0,9
OX40N,564802.0,9
XY25,482575.0,8
SR,274521.0,2
X5K,111746.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3410099.0,24
M-SC,1269680.0,15
X-MC,1225074.0,14
X-LC,1101595.0,13
X-SC,572366.0,6
H-LC,296907.0,3
M-LC,283417.0,4
L-SC,260170.0,3
H-MC,259721.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1199878.0      6
           AR         840038.0      5
           XR         789012.0      7
M-SC       XY24       734447.0      6
X-MC       XY24       577315.0      4
X-LC       X40        469041.0      5
X-SC       X40N       318709.0      4
X-LC       XY24       315513.0      2
X-MC       X40        302881.0      6
H-SC       SR         274521.0      2
           OX40N      250528.0      3
X-LC       X40N       234336.0      3
H-LC       AR         222135.0      2
X-MC       XY25       188143.0      2
H-MC       XY24       183821.0      1
L-SC       XR         169714.0      2
X-SC       XY24       166838.0      1
X-MC       X40N       156735.0      2
M-MC       XY24       155159.0      1
M-SC       OX40N      147918.0      4
           XR         137390.0      2
           XY25       126032.0      1
           AR         123893.0      2
M-LC       XR         113467.0      1
           X5K        111746.0      1
L-SC       OX40N       90456.0      1
X-SC       X40         86819.0      1
X-LC       XY25        82705.0      3
H-MC       OX40N       75900.0      1
H-LC       X200        74772.0      1
L-MC       XR          57552.0      1
H-SC       MH          56122.0      1
M-LC       XY25        44874.0      1
L-LC       XY25        40821.0      1
M-LC       XY24        13330.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
